In [ ]:
!pip3 install accelerate
!pip3 install huggingface-hub
!pip3 install safetensors
!pip3 install torch
!pip3 install transformers

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
input_dir = Path("/xdisk/josorio1/aconverse/UN_data/output/")

relevant_irrelevant = pd.read_csv(input_dir / "relevant_irrelevant.csv")
relevant_irrelevant["relevant"] = relevant_irrelevant["label"]

matconf = pd.read_csv(input_dir / "mat_conf.csv")
matconf["matconf"] = matconf["label"]
matconf = matconf[["id","matconf"]]

matcoop = pd.read_csv(input_dir / "mat_coop.csv")
matcoop["matcoop"] = matcoop["label"]
matcoop = matcoop[["id","matcoop"]]

verconf = pd.read_csv(input_dir / "ver_conf.csv")
verconf["verconf"] = verconf["label"]
verconf = verconf[["id","verconf"]]

vercoop = pd.read_csv(input_dir / "ver_coop.csv")
vercoop["vercoop"] = vercoop["label"]
vercoop = vercoop[["id","vercoop"]]

full_df = relevant_irrelevant.merge(matconf, on="id")
full_df = full_df.merge(matcoop, on="id")
full_df = full_df.merge(verconf, on="id")
full_df = full_df.merge(vercoop, on="id")

full_df.drop(columns=["label"], inplace=True)

full_df.to_csv(input_dir / "full_labeled.csv", index=False)

In [ ]:
full_df

In [ ]:
irrelevant = full_df[full_df["relevant"] == 0]
matconf = full_df[full_df["matconf"] == 1]
matcoop = full_df[full_df["matcoop"] == 1]
verconf = full_df[full_df["verconf"] == 1]
vercoop = full_df[full_df["vercoop"] == 1]

already_annotated = pd.read_csv("/xdisk/josorio1/aconverse/UN_data/data_master.csv")

min_category = min(len(df) for df in [irrelevant, matconf, matcoop, verconf, vercoop])

balanced_df = pd.concat([irrelevant[:min_category],
                         matconf[:min_category],
                         matcoop[:min_category],
                         verconf[:min_category],
                         vercoop[:min_category]])

balanced_df = balanced_df.drop_duplicates(subset="id", keep=False)

print(len(balanced_df))

balanced_df = balanced_df[~balanced_df["id"].isin(already_annotated["id"])]
balanced_df = balanced_df.sample(frac=1)

balanced_df = balanced_df.drop_duplicates(subset="id")

balanced_df.to_csv(input_dir / "balanced.csv", index=False)

balanced_df.drop(columns=["relevant", "matconf", "matcoop", "verconf", "vercoop", "Unnamed: 0"], inplace=True)

print(len(balanced_df))
balanced_df.to_csv(input_dir / "balanced_hidden_labels.csv", index=False)

In [ ]:
batch_num = 1
for i in range(180, len(balanced_df), 180):
    balanced_df.iloc[i-180:i].to_csv(input_dir / f"batches/batch_{batch_num}.csv", index=False)
    batch_num += 1